In [22]:
#import libs
import pandas as pd
from pandasql import sqldf
import numpy as np

In [19]:
#read in data
weather=pd.read_csv('Weather.csv')
weather.head()

,pickup_datetime,tempm,tempi,dewptm,dewpti,hum,wspdm,wspdi,wgustm,wgusti,...,precipm,precipi,conds,icon,fog,rain,snow,hail,thunder,tornado
0,2015-12-31 00:15:00,7.8,46.0,6.1,43.0,89.0,7.4,4.6,NaN,NaN,...,0.5,0.02,Light Rain,rain,0,1,0,0,0,0
1,2015-12-31 00:42:00,7.8,46.0,6.1,43.0,89.0,7.4,4.6,NaN,NaN,...,0.8,0.03,Overcast,cloudy,0,0,0,0,0,0
2,2015-12-31 00:51:00,7.8,46.0,6.1,43.0,89.0,5.6,3.5,NaN,NaN,...,0.8,0.03,Overcast,cloudy,0,0,0,0,0,0
3,2015-12-31 01:51:00,7.2,45.0,5.6,42.1,90.0,7.4,4.6,NaN,NaN,...,0.3,0.01,Overcast,cloudy,0,0,0,0,0,0
4,2015-12-31 02:51:00,7.2,45.0,5.6,42.1,90.0,0.0,0.0,NaN,NaN,...,NaN,NaN,Overcast,cloudy,0,0,0,0,0,0


In [18]:
#check original columns
weather.columns

Index(['pickup_datetime', 'tempm', 'tempi', 'dewptm', 'dewpti', 'hum', 'wspdm',
       'wspdi', 'wgustm', 'wgusti', 'wdird', 'wdire', 'vism', 'visi',
       'pressurem', 'pressurei', 'windchillm', 'windchilli', 'heatindexm',
       'heatindexi', 'precipm', 'precipi', 'conds', 'icon', 'fog', 'rain',
       'snow', 'hail', 'thunder', 'tornado'],
      dtype='object')

In [20]:
#fill na with 0
weather=weather.fillna(0)
#extract date and hour for processing
weather['date'] = weather['pickup_datetime'].str.extract('(\d{4}\-\d\d\-\d\d)', expand=True)
weather['hour'] = weather['pickup_datetime'].str.extract('(\d{2}:)', expand=True)
weather['hour'] = weather['hour'].str.extract('(\d{2})', expand=True)
#drop conditions besides rain, which is to be predicted
weather = weather.drop(columns=['conds','icon','fog', 'snow', 'hail',
       'thunder', 'tornado','pickup_datetime'])
weather.head()

,tempm,tempi,dewptm,dewpti,hum,wspdm,wspdi,wgustm,wgusti,wdird,...,pressurei,windchillm,windchilli,heatindexm,heatindexi,precipm,precipi,rain,date,hour
0,7.8,46.0,6.1,43.0,89.0,7.4,4.6,0.0,0.0,40,...,30.07,6.6,43.9,0.0,0.0,0.5,0.02,1,2015-12-31,00
1,7.8,46.0,6.1,43.0,89.0,7.4,4.6,0.0,0.0,0,...,30.06,6.6,43.9,0.0,0.0,0.8,0.03,0,2015-12-31,00
2,7.8,46.0,6.1,43.0,89.0,5.6,3.5,0.0,0.0,20,...,30.04,7.1,44.8,0.0,0.0,0.8,0.03,0,2015-12-31,00
3,7.2,45.0,5.6,42.1,90.0,7.4,4.6,0.0,0.0,0,...,30.02,5.9,42.6,0.0,0.0,0.3,0.01,0,2015-12-31,01
4,7.2,45.0,5.6,42.1,90.0,0.0,0.0,0.0,0.0,0,...,30.03,0.0,0.0,0.0,0.0,0.0,0.00,0,2015-12-31,02


In [21]:
#check remaining columns after cleaning
weather.columns

Index(['tempm', 'tempi', 'dewptm', 'dewpti', 'hum', 'wspdm', 'wspdi', 'wgustm',
       'wgusti', 'wdird', 'wdire', 'vism', 'visi', 'pressurem', 'pressurei',
       'windchillm', 'windchilli', 'heatindexm', 'heatindexi', 'precipm',
       'precipi', 'rain', 'date', 'hour'],
      dtype='object')

In [23]:
#summarize the data by date and hour, taking average of all numerical values. 
pv=pd.pivot_table(weather,index=['date','hour'],values=['tempm', 'tempi', 'dewptm', 'dewpti', 'hum', 'wspdm',
       'wspdi', 'wgustm', 'wgusti', 'wdird', 'wdire', 'vism', 'visi',
       'pressurem', 'pressurei', 'windchillm', 'windchilli', 'heatindexm',
       'heatindexi', 'precipm', 'precipi', 'rain'],aggfunc='mean')
df = pv.reset_index().round(decimals=3)
#fix the "rain" column = if it rained in the hour (resulting in mean value >0), we code it as 1
df['rain']= np.where(df['rain'] >0 , 1, 0)
#print results
df.head()

,date,hour,dewpti,dewptm,heatindexi,heatindexm,hum,precipi,precipm,pressurei,...,tempm,visi,vism,wdird,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm
0,2015-12-31,00,43.00,6.1,0.0,0.0,89.0,0.027,0.7,30.057,...,7.800,3.833,6.133,20.000,0.0,0.0,44.200,6.767,4.233,6.800
1,2015-12-31,01,42.10,5.6,0.0,0.0,90.0,0.010,0.3,30.020,...,7.200,8.000,12.900,0.000,0.0,0.0,42.600,5.900,4.600,7.400
2,2015-12-31,02,42.10,5.6,0.0,0.0,90.0,0.000,0.0,30.030,...,7.200,8.000,12.900,0.000,0.0,0.0,0.000,0.000,0.000,0.000
3,2015-12-31,03,41.00,5.0,0.0,0.0,87.0,0.000,0.0,30.023,...,7.033,8.333,13.433,203.333,0.0,0.0,28.033,3.733,3.067,4.933
4,2015-12-31,04,41.55,5.3,0.0,0.0,88.0,0.000,0.0,30.020,...,7.200,8.000,12.900,275.000,0.0,0.0,43.500,6.400,3.500,5.600


In [25]:
#delete days that don't have 24 hours
pysqldf = lambda q: sqldf(q, globals())

output= pysqldf('''
with cte as (select
date,
count(distinct hour) as entries
from 
df
group by 1
having entries = 24)

select
*
from 
df
where
date in (select date from cte)
''')

In [27]:
#sort index based on date and hour
output = output.sort_values(["date", "hour"], ascending = (True, True))
#consolidate date hour back for time series
output['date_hour']=output.date.astype('str')+' '+output.hour.astype('str')+':00:00'
#typecast date hour
output['date_hour']=pd.to_datetime(output['date_hour'].astype(str))
#reset index
output.set_index('date_hour',inplace=True)
#drop date and hour
output = output.drop(columns=['date','hour'])
output.head()

,dewpti,dewptm,heatindexi,heatindexm,hum,precipi,precipm,pressurei,pressurem,rain,...,tempm,visi,vism,wdird,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm
date_hour,,,,,,,,,,,,,,,,,,,,,
2015-12-31 00:00:00,43.00,6.1,0.0,0.0,89.0,0.027,0.7,30.057,1017.667,1,...,7.800,3.833,6.133,20.000,0.0,0.0,44.200,6.767,4.233,6.800
2015-12-31 01:00:00,42.10,5.6,0.0,0.0,90.0,0.010,0.3,30.020,1016.500,0,...,7.200,8.000,12.900,0.000,0.0,0.0,42.600,5.900,4.600,7.400
2015-12-31 02:00:00,42.10,5.6,0.0,0.0,90.0,0.000,0.0,30.030,1016.700,0,...,7.200,8.000,12.900,0.000,0.0,0.0,0.000,0.000,0.000,0.000
2015-12-31 03:00:00,41.00,5.0,0.0,0.0,87.0,0.000,0.0,30.023,1016.633,0,...,7.033,8.333,13.433,203.333,0.0,0.0,28.033,3.733,3.067,4.933
2015-12-31 04:00:00,41.55,5.3,0.0,0.0,88.0,0.000,0.0,30.020,1016.500,0,...,7.200,8.000,12.900,275.000,0.0,0.0,43.500,6.400,3.500,5.600


In [28]:
#output to csv
output.to_csv('working_data_new.csv',index=True)
